<a href="https://colab.research.google.com/github/yoohw0317/MILAB/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multivariate Linear Regression**

여러개의 정보를 통해서 하나의 결과를 도출해내는 학습법



matmul()로 한번에 계산
더 간결하고, x의의 길이가 바뀌어도 코드를 바꿀 필요가 없고 속도도 더 빠름.

cost function의 수식은 비슷함.

W : = W -a * d/dx a


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# 데이터 (여기가 simple regression과 달라짐)
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화 (여기도 약간 달라짐)
W = torch.zeros((3,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

In [ ]:
nb_epochs = 20
for epoch in range(nb_epochs+1):

  #H(x)를 계산)
  hypothesis = x_train.matmul(W) + b

  #Cost 계산, Cost는 점점 작아지도록 됨
  cost = torch.mean((hypothesis - y_train) ** 2)

  #cost로 H(x)를 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print(f'Epoch {epoch:4d}/{nb_epochs} hypothesis: {hypothesis.squeeze().detach()} Cost: {cost.item():.6f}')

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7967, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936096
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371063
Epoch    6/20 hypothesis: tensor([148.1035, 178.0143, 175.3980, 191.0042, 135.7812]) Cost: 29.758249
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8509, 193.6753, 137.6805]) Cost: 10.445267
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391237
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493121
Epoch   10/20 hypo

# nn.Module 사용하기

* nn.Module 을 상속해서 모델을 생성
* nn.Linear (3, 1)
  * 입력 차원: 3
  * 출력 차원: 1
* Hypothesis 계산은 forward()에서 처리됨
* Gradient 계산은 PyTorch가 알아서 해줌 (backward() 통해서)



In [ ]:
class MultivariateLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1)

  def forward(self, x):
    return self.linear(x)

# F.mse_loss


*   torch.nn.functional 에서 제공하는 loss function 사용
*   쉽게 다른 loss와 교체 가능 (l1_loss, smooth_l1_loss 등등)



In [ ]:
# 데이터 (위의 코드와 변화 없음)
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화 - 매우 간결해짐
model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer_new = optim.SGD(model.parameters(), lr=1e-5)

In [ ]:
nb_epochs = 50
for epoch in range(nb_epochs+1):

  #H(x)를 계산 - model을 이용해서 계산하게 됨.
  Hypothesis_new = model(x_train)

  #Cost 계산, Cost는 점점 작아지도록 됨
  cost_new = F.mse_loss(Hypothesis_new, y_train)

  #cost로 H(x)를 개선
  optimizer_new.zero_grad()
  cost_new.backward()
  optimizer_new.step()

  print(f'Epoch {epoch:4d}/{nb_epochs} hypothesis: {Hypothesis_new.squeeze().detach()} Cost: {cost_new.item():.6f}')

Epoch    0/50 hypothesis: tensor([4.8076, 9.0994, 7.2267, 7.6482, 8.0502]) Cost: 27175.240234
Epoch    1/50 hypothesis: tensor([69.1865, 86.4780, 83.4692, 90.6744, 67.0705]) Cost: 8518.154297
Epoch    2/50 hypothesis: tensor([105.2300, 129.7993, 126.1546, 137.1577, 100.1138]) Cost: 2670.148926
Epoch    3/50 hypothesis: tensor([125.4095, 154.0533, 150.0526, 163.1820, 118.6135]) Cost: 837.110168
Epoch    4/50 hypothesis: tensor([136.7072, 167.6323, 163.4322, 177.7520, 128.9708]) Cost: 262.549255
Epoch    5/50 hypothesis: tensor([143.0325, 175.2346, 170.9230, 185.9093, 134.7694]) Cost: 82.455276
Epoch    6/50 hypothesis: tensor([146.5738, 179.4908, 175.1168, 190.4762, 138.0158]) Cost: 26.005285
Epoch    7/50 hypothesis: tensor([148.5565, 181.8737, 177.4648, 193.0331, 139.8333]) Cost: 8.311244
Epoch    8/50 hypothesis: tensor([149.6666, 183.2077, 178.7794, 194.4646, 140.8508]) Cost: 2.765025
Epoch    9/50 hypothesis: tensor([150.2881, 183.9546, 179.5154, 195.2660, 141.4204]) Cost: 1.026589

# Data in the Real World: Problem

*   엄청난 양의 데이터를 한 번에 학습시키기는 불가능, 느리고 하드웨어적으로 불가능
*   일부분의 데이터로만 학습시키는 방법

# Minibatch Gradient Descent
*   전체 데이터를 균일하게 나눠서 학습하는 방법

In [ ]:
# torch.utils.data.Dataset 상속
from torch.utils.data import Dataset 

class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]

  # 이 데이터셋의 총 데이터 수
  def __len__(self):
    return len(self.x_data)
  
  #어떠한 인덱스 idx를 입력받았을 때, 그에 상응하는ㄷ 입출력 데이터 반환
  def __getitem__ (self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y

dataset = CustomDataset()

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset,
    # 각 minibatch의 크기를 의미, 통상적으로 2의 제곱수로 설정한다.
    batch_size=2,
    # Epoch 마다 데이터 셋을 섞어서 데이터가 학습되는 순서를 바꾼다.
    shuffle=True,
)

In [ ]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        
        prediction = model(x_train)
        cost = F.mse_loss(prediction, y_train)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print(f'Epoch{epoch:4d}/{nb_epochs} Batch{batch_idx+1}/{len(dataloader)} Cost: {cost.item():.6f}')

Epoch   0/20 Batch1/3 Cost: 0.107265
Epoch   0/20 Batch2/3 Cost: 0.431069
Epoch   0/20 Batch3/3 Cost: 0.081811
Epoch   1/20 Batch1/3 Cost: 0.431069
Epoch   1/20 Batch2/3 Cost: 0.045696
Epoch   1/20 Batch3/3 Cost: 0.204949
Epoch   2/20 Batch1/3 Cost: 0.425774
Epoch   2/20 Batch2/3 Cost: 0.112561
Epoch   2/20 Batch3/3 Cost: 0.081811
Epoch   3/20 Batch1/3 Cost: 0.045696
Epoch   3/20 Batch2/3 Cost: 0.523458
Epoch   3/20 Batch3/3 Cost: 0.020172
Epoch   4/20 Batch1/3 Cost: 0.014877
Epoch   4/20 Batch2/3 Cost: 0.461889
Epoch   4/20 Batch3/3 Cost: 0.204949
Epoch   5/20 Batch1/3 Cost: 0.461889
Epoch   5/20 Batch2/3 Cost: 0.107265
Epoch   5/20 Batch3/3 Cost: 0.020172
Epoch   6/20 Batch1/3 Cost: 0.045696
Epoch   6/20 Batch2/3 Cost: 0.431069
Epoch   6/20 Batch3/3 Cost: 0.204949
Epoch   7/20 Batch1/3 Cost: 0.523458
Epoch   7/20 Batch2/3 Cost: 0.050992
Epoch   7/20 Batch3/3 Cost: 0.009581
Epoch   8/20 Batch1/3 Cost: 0.107265
Epoch   8/20 Batch2/3 Cost: 0.431069
Epoch   8/20 Batch3/3 Cost: 0.081811
E